In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard imports
import numpy as np
import xarray as xr
import tqdm as tqdm

In [3]:
# Import OpenSense modules as submodules
import sys
import os

sys.path.append(os.path.abspath("./pycomlink/"))
sys.path.append(os.path.abspath("./poligrain/src/"))
sys.path.append(os.path.abspath("./mergeplg/src/"))

import pycomlink as pycml 
import poligrain as plg
import mergeplg 

# OpenMRG

In [4]:
# OpenMRG
ds_rad = xr.open_dataset("data/andersson_2022_OpenMRG/radar/openmrg_rad.nc")                    
ds_cmls = xr.open_dataset("data/processed_cml_OpenMRG.nc").R_acc                                  

# Adjust rainfall fields

In [5]:
os.makedirs('data/adjusted_fields', exist_ok=True)
months = ['2015-06', '2015-07', '2015-08']

In [6]:
# additive IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_add_IDW']=xr.where(
    np.isnan(data['rainfall_add_IDW']), 0, data['rainfall_add_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_IDW.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [00:46<00:00, 61.87it/s]


month: 2015-07


100%|██████████| 2976/2976 [00:46<00:00, 64.63it/s]


month: 2015-08


100%|██████████| 2976/2976 [00:45<00:00, 65.39it/s]


In [7]:
# multiplicative IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_mul_IDW']=xr.where(
    np.isnan(data['rainfall_mul_IDW']), 0, data['rainfall_mul_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_mul_IDW.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [00:44<00:00, 65.31it/s]


month: 2015-07


100%|██████████| 2976/2976 [00:46<00:00, 63.76it/s]


month: 2015-08


100%|██████████| 2976/2976 [00:46<00:00, 64.40it/s]


In [8]:
# additive POINT BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_pBK']=xr.where(
    np.isnan(data['rainfall_add_pBK']), 0, data['rainfall_add_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_pBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [01:17<00:00, 37.18it/s]


month: 2015-07


100%|██████████| 2976/2976 [01:32<00:00, 32.20it/s]


month: 2015-08


100%|██████████| 2976/2976 [01:21<00:00, 36.58it/s]


In [9]:
# multiplicative POINT BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_pBK']=xr.where(
    np.isnan(data['rainfall_mul_pBK']), 0, data['rainfall_mul_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_mul_pBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [01:12<00:00, 39.57it/s]


month: 2015-07


100%|██████████| 2976/2976 [01:22<00:00, 36.14it/s]


month: 2015-08


100%|██████████| 2976/2976 [01:13<00:00, 40.59it/s]


In [10]:
# additive LINE BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_lBK']=xr.where(
    np.isnan(data['rainfall_add_lBK']), 0, data['rainfall_add_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_lBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [01:19<00:00, 36.35it/s]


month: 2015-07


100%|██████████| 2976/2976 [01:23<00:00, 35.73it/s]


month: 2015-08


100%|██████████| 2976/2976 [01:12<00:00, 40.87it/s]


In [11]:
# multiplicative LINE BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_lBK']=xr.where(
    np.isnan(data['rainfall_mul_lBK']), 0, data['rainfall_mul_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_mul_lBK.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [01:06<00:00, 43.60it/s]


month: 2015-07


100%|██████████| 2976/2976 [01:24<00:00, 35.15it/s]


month: 2015-08


100%|██████████| 2976/2976 [01:16<00:00, 38.77it/s]


In [12]:
# KED
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_ked": xr.concat(rainfall, dim="time") })
data['rainfall_ked']=xr.where(
    np.isnan(data['rainfall_ked']), 0, data['rainfall_ked']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_ked.nc')    
del data, merger

month: 2015-06


100%|██████████| 2879/2879 [01:15<00:00, 38.22it/s]


month: 2015-07


100%|██████████| 2976/2976 [01:31<00:00, 32.68it/s]


month: 2015-08


100%|██████████| 2976/2976 [01:15<00:00, 39.63it/s]


# OpenRainER

In [4]:
# OpenRainER
ds_rad = xr.open_dataset("data/covi_2024_OpenRainER/openrainer_radar.nc")         
ds_cmls = xr.open_dataset("data/processed_cml_OpenRainER.nc").R_acc                     

# Adjust rainfall fields

In [5]:
os.makedirs('data/adjusted_fields', exist_ok=True)
months = ['2022-06', '2022-07', '2022-08']

In [15]:
# additive IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_add_IDW']=xr.where(
    np.isnan(data['rainfall_add_IDW']), 0, data['rainfall_add_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_IDW.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [02:55<00:00, 16.38it/s]


month: 2022-07


100%|██████████| 2976/2976 [03:05<00:00, 16.05it/s]


month: 2022-08


100%|██████████| 2976/2976 [03:16<00:00, 15.16it/s]


In [6]:
# multiplicative IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        multiplicative_factors=(0.1, 2),
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_mul_IDW']=xr.where(
    np.isnan(data['rainfall_mul_IDW']), 0, data['rainfall_mul_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_IDW.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [02:25<00:00, 19.76it/s]


month: 2022-07


100%|██████████| 2976/2976 [02:39<00:00, 18.70it/s]


month: 2022-08


100%|██████████| 2976/2976 [02:56<00:00, 16.86it/s]


In [17]:
# additive POINT BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_pBK']=xr.where(
    np.isnan(data['rainfall_add_pBK']), 0, data['rainfall_add_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_pBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [23:56<00:00,  2.00it/s] 


month: 2022-07


100%|██████████| 2976/2976 [07:42<00:00,  6.44it/s]


month: 2022-08


100%|██████████| 2976/2976 [19:03<00:00,  2.60it/s] 


In [18]:
# multiplicative POINT BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_pBK']=xr.where(
    np.isnan(data['rainfall_mul_pBK']), 0, data['rainfall_mul_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_pBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [14:11<00:00,  3.38it/s] 


month: 2022-07


100%|██████████| 2976/2976 [05:49<00:00,  8.51it/s]


month: 2022-08


100%|██████████| 2976/2976 [13:59<00:00,  3.54it/s]


In [19]:
# additive LINE BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_lBK']=xr.where(
    np.isnan(data['rainfall_add_lBK']), 0, data['rainfall_add_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_lBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [23:53<00:00,  2.01it/s] 


month: 2022-07


100%|██████████| 2976/2976 [07:37<00:00,  6.50it/s]


month: 2022-08


100%|██████████| 2976/2976 [19:23<00:00,  2.56it/s]


In [20]:
# multiplicative LINE BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_mul_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_mul_lBK']=xr.where(
    np.isnan(data['rainfall_mul_lBK']), 0, data['rainfall_mul_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_mul_lBK.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [14:15<00:00,  3.37it/s] 


month: 2022-07


100%|██████████| 2976/2976 [05:29<00:00,  9.04it/s]


month: 2022-08


100%|██████████| 2976/2976 [14:01<00:00,  3.54it/s]


In [21]:
# KED
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_ked": xr.concat(rainfall, dim="time") })
data['rainfall_ked']=xr.where(
    np.isnan(data['rainfall_ked']), 0, data['rainfall_ked']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_ked.nc')    
del data, merger

month: 2022-06


100%|██████████| 2880/2880 [20:50<00:00,  2.30it/s] 


month: 2022-07


100%|██████████| 2976/2976 [04:55<00:00, 10.06it/s] 


month: 2022-08


100%|██████████| 2976/2976 [16:13<00:00,  3.06it/s] 
